<a href="https://colab.research.google.com/github/JRI11930/IPN-Analysis/blob/main/IPN_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Proyecto DM

## Importación de librerías

In [ ]:
! pip install gspread

In [ ]:
#LIBRERÍAS BÁSICAS
import pandas as pd
import numpy as np

#EDA


#PREPROCESAMIENTO


#MODELADO

#UTILS
import gspread

## Carga de Datos

In [ ]:
sa = gspread.service_account(filename='service_account.json')
sh = sa.open('ProyectoDM')

In [ ]:
worksheet = sh.sheet1
data = worksheet.get_all_records()
df = pd.DataFrame(data)

## EDA

In [ ]:
df.head()

,Marca temporal,Dirección de correo electrónico,¿A qué escuela perteneces?,¿Cuál es tu promedio global?,¿Cuántas materias has reprobado a lo largo de tu trayectoria?,¿Qué edad tienes?,¿Cuál es tu género?,¿Tienes dependientes económicos?,¿Tienes personas a tu cuidado?,¿Te sientes apoyado por tu familia?,...,¿Eres foráneo?,¿Cuánto tiempo tardas en llegar a tu escuela en minutos?,¿Cuántas horas diarias dedicas al estudio fuera de clases?,Para validar la lectura de esta encuesta selecciona la opción 3 horas,"¿Cuántas horas trabajas al día? \nSi no trabajas, escribe '0'.","¿Padeces de alguna enfermedad crónica (Por ejemplo: diabetes, hipertensión, asma, etc.)?",¿Te gusta tu carrera?,¿Piensas ejercer la carrera?,¿Cuántas horas duermes en promedio al día?,¿Con qué frecuencia utilizas herramientas de IA?
0,28/11/2024 11:44:10,ax3l.riv@gmail.com,ESCOM,8.4,0.0,20,Masculino,No,No,5,...,Sí,15.0,3.0,3 horas,0.0,No,Sí,Sí,6.0,4
1,28/11/2024 12:18:18,bejarjesus032@gmail.com,ESCOM,6.0,1.0,19,Masculino,Sí,No,5,...,No,60.0,1.0,3 horas,0.0,No,Sí,Sí,7.0,4
2,28/11/2024 12:18:44,concepciondaniel90@gmail.com,ESCOM,7.7,0.0,19,Masculino,No,No,5,...,No,60.0,1.0,3 horas,5.0,No,Sí,Sí,5.0,4
3,28/11/2024 12:18:57,hernandezakira33@gmail.com,ESCOM,6.4,1.0,19,Masculino,No,No,5,...,No,60.0,3.0,3 horas,1.0,No,Indiferente,Sí,5.0,1
4,28/11/2024 12:19:02,betohacker033@gmail.com,ESCOM,6.5,4.0,21,Masculino,No,No,5,...,No,40.0,4.0,3 horas,0.0,No,Sí,Sí,6.0,3
